Prepare excel file with metadata from adatas for an easier overview of the availiable samples.

In [97]:
import scanpy as sc
import pandas as pd
import os
import pickle

In [2]:
path_rna='/lustre/groups/ml01/workspace/karin.hrovatin/data/pancreas/scRNA/'

In [99]:
# All datasets
datasets=[
    ('human','GSE83139','GSE83139/GEO/'),
    ('human','GSE154126','GSE154126/GEO/'),
    ('human','GSE101207','GSE101207/GEO/'),
    ('human','GSE124742_GSE164875_patch','GSE124742_GSE164875/GEO/patch/'),
    ('human','GSE124742_FACS','GSE124742_GSE164875/GEO/FACS/'),
    ('human','GSE86469','GSE86469/GEO/'),
    ('human','GSE81547','GSE81547/GEO/'),
    ('human','GSE198623','P21000/sophie/human/'),
    ('human','GSE81608','GSE81608/GEO/'),
    ('human','GSE148073','GSE148073/GEO/'),
    ('mouse','GSE137909','GSE137909/GEO/'),
    ('mouse','GSE83146','GSE83146/GEO/')
]

In [100]:
# Save info about all datasets
pickle.dump(datasets,open(path_rna+'external_info.pkl','wb'))

In [101]:
# Save metadata of all datasets
writer = pd.ExcelWriter(path_rna+'external_metadata.xlsx',
                        engine='xlsxwriter') 
for species,name,d in datasets:
    print(species,name)
    # Load adata - filtered if exists
    path_filtered=path_rna+d+'adata_filtered.h5ad'
    file=path_filtered if os.path.exists(path_filtered) else path_rna+d+'adata.h5ad'
    obs=sc.read(file,backed='r').obs.copy()
    donors_all=set(obs.donor.unique()) if 'donor' in obs.columns else set()
    # all data cols
    print('All cols:',list(obs.columns))
    # Remove cells not useful for saving and grouping
    cols_keep=[c for c in obs.columns 
         # Remove organ/organism/tisse/cell info/geo_accession 
         # (as not added to all samples now,may also not be sample specific (e.g. could be cells)
         if 'organ' not in c and 'tissue' not in c and 'cell_' not in c
            and 'geo_accession' not in c ]
    # Make groups based on kept cols
    # Make sure empty groups are dropped and groups with NA are not dsriopped - does not work
    # Thuis fill nan with NA, must first remove categorical
    def uncategorize(col):
        if col.dtype.name == 'category':
            return col.astype(col.cat.categories.dtype)         
        else:
            return col
    obs = obs.apply(uncategorize, axis=0)
    obs=obs.fillna('NA')
    obs=obs.groupby(cols_keep,observed=True,dropna=False)
    # N beta cells per group - add to df
    n_beta=obs.apply(lambda x:x.query('cell_type=="beta"').shape[0])
    # Workaround to add n_beta - remove size latter
    obs=pd.DataFrame(obs.size())
    obs['N_beta_cells']=n_beta.values
    obs=obs.reset_index().drop(0,axis=1)
    # Add species
    obs['organism']=species
    # Display and save
    print('Saved cols:',list(obs.columns))
    display(obs)
    # Make sure donor (if exists) is unique to sinle group - groupping worked as expected
    if 'donor' in obs.columns and obs.value_counts('donor').max()>1:
        raise ValueError('Duplicated donor')
    # make sure all donors are kept
    if len(donors_all)>0:
        if not donors_all==set(obs.donor.unique()):
            raise ValueError('Donors not matching')
    obs.to_excel(writer, sheet_name=name,index=False)   
writer.save()

human GSE83139
All cols: ['tissue', 'disease', 'cell_type', 'geo_accession', 'organ', 'organism', 'donor', 'age', 'sex', 'ethnicity', 'BMI', 'cultured_days', 'cell_type_original']
Saved cols: ['disease', 'donor', 'age', 'sex', 'ethnicity', 'BMI', 'cultured_days', 'N_beta_cells', 'organism']


,disease,donor,age,sex,ethnicity,BMI,cultured_days,N_beta_cells,organism
0,T1D,ACGI428,23 y,male,NA,25.0,10.0,6,human
1,T2D,HP-15041,57 y,male,african_american,23.98,4.0,5,human
2,T2D,HP-15085,37 y,female,white,39.3,4.0,16,human
3,T2D,HP-15085: cultured,37 y,female,white,39.3,12.0,17,human
4,healthy,AAJF122,52 y,male,asian,29.1,6.0,1,human
5,healthy,ABAF490,39 y,female,white,45.2,4.0,30,human
6,healthy,ACAP236,21 y,male,white,39.0,2.0,17,human
7,healthy,ICRH76,2 y,male,white,13.6,2.0,4,human
8,healthy,ICRH80,19 m,female,white,18.0,3.0,15,human


human GSE154126
All cols: ['geo_accession', 'disease', 'cell_type_original', 'donor', 'age', 'sex', 'ethnicity', 'BMI', 'cell_type']
Saved cols: ['disease', 'donor', 'age', 'sex', 'ethnicity', 'BMI', 'N_beta_cells', 'organism']


,disease,donor,age,sex,ethnicity,BMI,N_beta_cells,organism
0,T2D,ACIA085_APT2D,64 y,male,european_american,33.16,0,human
1,T2D,ADDU206,41 y,male,hispanic,26.2,1,human
2,T2D,AEBK009,53 y,male,NA,NA,2,human
3,T2D,AEGY230A,52 y,female,hispanic,42.8,5,human
4,T2D,AEHL151,59 y,male,hispanic,27.7,1,human
5,T2D,HP-15041,57 y,male,african_american,23.98,4,human
6,T2D,HP-15085-01T2D,37 y,female,european_american,39.3,27,human
7,T2D,HP15269-01T2D,55 y,female,european_american,29.84,5,human
8,T2D,HP16012_1T2D,42 y,male,european_american,43.7,1,human
9,T2D,ICRH95,60 y,female,african_american,28.2,0,human


human GSE101207
All cols: ['donor', 'cell_type', 'cell_type_original', 'sex', 'age', 'BMI', 'HbA1c', 'ethnicity', 'death_cause', 'disease', 'organ', 'tissue', 'organism']
Saved cols: ['donor', 'sex', 'age', 'BMI', 'HbA1c', 'ethnicity', 'death_cause', 'disease', 'N_beta_cells', 'organism']


...

,disease,donor,sex,age,ethnicity,BMI,years_diagnosis,HbA1c,N_beta_cells,organism
0,AAB,HPAP019,male,22 y,caucasian,29.8,NA,5.2,3193,human
1,AAB,HPAP024,male,18 y,caucasian,24.3,NA,5.5,89,human
2,AAB,HPAP029,male,23 y,caucasian,28.6,NA,5.3,30,human
3,AAB,HPAP038,male,13 y,caucasian,18.34,NA,5.7,1074,human
4,AAB,HPAP043,male,15 y,hispanic,24.07,NA,5.90,281,human
5,AAB,HPAP045,female,27 y,caucasian,26.2,NA,5.2,1066,human
6,AAB,HPAP049,male,29 y,caucasian,37.2,NA,5.4,959,human
7,AAB,HPAP050,female,21 y,hispanic,28.99,NA,5.1,377,human
8,T1D,HPAP020,male,14 y,caucasian,13.32,NA,NA,199,human
9,T1D,HPAP021,female,13 y,caucasian,21.4,7,NA,81,human


mouse GSE137909
All cols: ['strain', 'STZ', 'insulin_implant', 'time_after_STZ', 'age', 'batch', 'cell_type_original', 'cell_subtype_original', 'cell_type', 'donor', 'sex', 'disease']
Saved cols: ['strain', 'STZ', 'insulin_implant', 'time_after_STZ', 'age', 'batch', 'donor', 'sex', 'disease', 'N_beta_cells', 'organism']


,strain,STZ,insulin_implant,time_after_STZ,age,batch,donor,sex,disease,N_beta_cells,organism
0,Ngn3-Cre; Rosa-RFP,False,False,NA,12 d,1,Ngn3Cre_P12_1,male,healthy,61,mouse
1,Ngn3-Cre; Rosa-RFP,False,False,NA,12 d,2,Ngn3Cre_P12_2,male,healthy,67,mouse
2,Ngn3-Cre; Rosa-RFP,False,False,NA,2 m,1,Ngn3Cre_Control_1,male,healthy,210,mouse
3,Ngn3-Cre; Rosa-RFP,False,False,NA,21 d,1,Ngn3Cre_P21_1,male,healthy,105,mouse
4,Ngn3-Cre; Rosa-RFP,False,False,NA,3 d,1,Ngn3Cre_P3_1,male,healthy,123,mouse
5,Ngn3-Cre; Rosa-RFP,False,False,NA,4 m,2,Ngn3Cre_Control_2,male,healthy,138,mouse
6,Ngn3-Cre; Rosa-RFP,False,False,NA,4 m,3,Ngn3Cre_Control_3,male,healthy,21,mouse
7,Ngn3-Cre; Rosa-RFP,True,False,2 m,4 m,1,Ngn3Cre_STZ_M2_NoIns_1,male,T2D,5,mouse
8,Ngn3-Cre; Rosa-RFP,True,False,2 m,4 m,2,Ngn3Cre_STZ_M2_NoIns_2,male,T2D,11,mouse
9,Ngn3-Cre; Rosa-RFP,True,True,12 d,2.4 m,1,Ngn3Cre_STZ_D12_1,male,T2D,16,mouse


mouse GSE83146
All cols: ['geo_accession', 'age', 'sex', 'cell_type', 'disease']
Saved cols: ['age', 'sex', 'disease', 'N_beta_cells', 'organism']


,age,sex,disease,N_beta_cells,organism
0,26 m,male,healthy,69,mouse
1,3 m,male,healthy,138,mouse


In [82]:
path_rna+'external_metadata.xlsx'

'/lustre/groups/ml01/workspace/karin.hrovatin/data/pancreas/scRNA/external_metadata.xlsx'